In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix,accuracy_score

In [2]:
filepath = "Churn_Modelling.csv"

dataset = pd.read_csv(filepath)

In [3]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [5]:
X[0:5]

array([[619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 'Spain', 'Female', 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 'France', 'Female', 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [699, 'France', 'Female', 39, 1, 0.0, 2, 0, 0, 93826.63],
       [850, 'Spain', 'Female', 43, 2, 125510.82, 1, 1, 1, 79084.1]],
      dtype=object)

In [6]:
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [7]:
X[0:5]

array([[619, 'France', 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 'Spain', 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 'France', 0, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [699, 'France', 0, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [850, 'Spain', 0, 43, 2, 125510.82, 1, 1, 1, 79084.1]],
      dtype=object)

In [8]:
le.classes_

array(['Female', 'Male'], dtype=object)

In [9]:
print(le.transform(['Male']))
print(le.transform(['Female']))

[1]
[0]


In [10]:
print(le.inverse_transform([1]))
print(le.inverse_transform([0]))

['Male']
['Female']


In [11]:
ct = ColumnTransformer(transformers =[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [12]:
X[:5]

array([[1.0, 0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [0.0, 0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [1.0, 0.0, 0.0, 502, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [1.0, 0.0, 0.0, 699, 0, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [0.0, 0.0, 1.0, 850, 0, 43, 2, 125510.82, 1, 1, 1, 79084.1]],
      dtype=object)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 0) # Splitting data

In [14]:
sc = StandardScaler() # Feature Scaling

In [15]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Training ANN

In [16]:
ann = tf.keras.models.Sequential()

In [17]:
ann.add(tf.keras.layers.Dense(units=8, activation='relu'))
ann.add(tf.keras.layers.Dense(units=8, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [18]:
ann.fit(X_train,y_train, batch_size= 32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 661us/step - loss: 0.5775 - accuracy: 0.7300
Epoch 2/100
250/250 [==============================] - 0s 664us/step - loss: 0.4724 - accuracy: 0.8008
Epoch 3/100
250/250 [==============================] - 0s 696us/step - loss: 0.4429 - accuracy: 0.8086
Epoch 4/100
250/250 [==============================] - 0s 687us/step - loss: 0.4276 - accuracy: 0.8101
Epoch 5/100
250/250 [==============================] - 0s 825us/step - loss: 0.4157 - accuracy: 0.8150
Epoch 6/100
250/250 [==============================] - 0s 556us/step - loss: 0.4040 - accuracy: 0.8227
Epoch 7/100
250/250 [==============================] - 0s 619us/step - loss: 0.3918 - accuracy: 0.8334
Epoch 8/100
250/250 [==============================] - 0s 626us/step - loss: 0.3803 - accuracy: 0.8420
Epoch 9/100
250/250 [==============================] - 0s 617us/step - loss: 0.3693 - accuracy: 0.8469
Epoch 10/100
250/250 [==============================] - 0s 643us/step - l

Prediction

In [25]:
test_case = np.array([[600, 'France', 'Male', 40, 3, 60000, 2, 1, 1, 50000]])    # manually have to set numpy array as it is the only array which supports slicing with tuple which we'll use in LabelEncoder
test_case[:,2] = le.transform(test_case[:,2])    # Using LabelEncoder to convert Male/Female in 1/0 format
test_case = ct.transform(test_case)    # Using OneHotEncoder to convert countries in binary format
test_case = sc.transform(test_case)    # Feature Scaling our test case
print('Prediction : ',ann.predict(test_case))
print('Should we say goodbye to this customer ? :','YES' if ann.predict([test_case]) > 0.5 else 'NO') # If predicted value is > 0.5, then customer leaves the bank, else No

Prediction :  [[0.0379841]]
Should we say goodbye to this customer ? : NO


In [20]:
y_pred = ann.predict(X_test)
y_pred

array([[0.2494942 ],
       [0.38138732],
       [0.10110426],
       ...,
       [0.1845175 ],
       [0.23494455],
       [0.26284117]], dtype=float32)

In [21]:
y_pred = (y_pred > 0.5).astype(int)
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [22]:
np.concatenate((y_pred.reshape(-1,1),y_test.reshape(-1,1)),1)

array([[0, 0],
       [0, 1],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]], dtype=int64)

Confusion matrix & Accuracy score

In [23]:
cm = confusion_matrix(y_test,y_pred)
cm

array([[1507,   88],
       [ 189,  216]], dtype=int64)

In [24]:
accuracy_score(y_test,y_pred)

0.8615